In [1]:
!pip install datasets transformers tensorflow numpy matplotlib opencv-python scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from PIL import Image
from datasets import load_dataset

# تحميل مجموعة البيانات
dataset = load_dataset("yasimed/lung_cancer")

# دالة استخراج التصنيفات من النصوص
def extract_label(text):
    return 1 if "cancer" in text.lower() else 0  # تحويل النص إلى تصنيف رقمي

# استخراج الصور والتصنيفات مع التأكد من الاتساق
images = []
labels = []

for item in dataset['train']:
    # تحويل الصورة إلى كائن PIL
    image = item['image'].convert("RGB")  # تحويل الصورة إلى 3 قنوات (RGB)

    # تغيير الحجم إلى 224x224
    image = image.resize((224, 224))

    # تحويل الصورة إلى مصفوفة NumPy وتطبيع القيم بين 0 و 1
    image = np.array(image, dtype=np.float32) / 255.0

    images.append(image)
    labels.append(extract_label(item['text']))  # استخراج التصنيف من النص

# تحويل القوائم إلى مصفوفات NumPy بعد التأكد من التجانس
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

# التحقق من الأشكال
print("✅ شكل الصور:", images.shape)  # يجب أن يكون (613, 224, 224, 3)
print("✅ شكل التصنيفات:", labels.shape)  # يجب أن يكون (613,)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/613 [00:00<?, ? examples/s]

✅ شكل الصور: (613, 224, 224, 3)
✅ شكل التصنيفات: (613,)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [5]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# طباعة حجم البيانات بعد التقسيم
print("✅ عدد صور التدريب:", X_train.shape[0])
print("✅ عدد صور الاختبار:", X_test.shape[0])


✅ عدد صور التدريب: 490
✅ عدد صور الاختبار: 123


In [6]:
# إنشاء محول بيانات التدريب
train_datagen = ImageDataGenerator(
    rotation_range=20,  # تدوير عشوائي حتى 20 درجة
    width_shift_range=0.1,  # إزاحة أفقية بنسبة 10%
    height_shift_range=0.1,  # إزاحة رأسية بنسبة 10%
    horizontal_flip=True,  # قلب الصورة أفقيًا
)

# تجهيز البيانات للتدريب والاختبار
train_generator = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=32, shuffle=False)


In [7]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# تحميل VGG19 بدون الطبقات العليا
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# تجميد الطبقات الأصلية
base_model.trainable = False

# إضافة الطبقات المخصصة
x = GlobalAveragePooling2D()(base_model.output)  # تقليل الأبعاد
x = Dense(256, activation='relu')(x)  # طبقة مخفية مع تنشيط ReLU
x = Dropout(0.5)(x)  # تقليل فرط التكيف (Overfitting)
output = Dense(1, activation='sigmoid')(x)  # مخرجات ثنائية (سرطان أو لا)

# إنشاء النموذج
model = Model(inputs=base_model.input, outputs=output)

# تجميع النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# عرض ملخص النموذج
model.summary()

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 20,155,969 (76.89 MB)

 Trainable params: 131,585 (514.00 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [8]:
# تدريب النموذج
history = model.fit(train_generator, validation_data=test_generator, epochs=40)

# حفظ النموذج بعد التدريب
model.save("VGG-19.h5")


Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.7289 - loss: 0.5566 - val_accuracy: 0.8943 - val_loss: 0.3632
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 293ms/step - accuracy: 0.8373 - loss: 0.3521 - val_accuracy: 0.9431 - val_loss: 0.3361
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 307ms/step - accuracy: 0.8684 - loss: 0.3084 - val_accuracy: 0.9024 - val_loss: 0.2469
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 397ms/step - accuracy: 0.9209 - loss: 0.2177 - val_accuracy: 0.9024 - val_loss: 0.2148
Epoch 5/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.9427 - loss: 0.1753 - val_accuracy: 0.9512 - val_loss: 0.2188
Epoch 6/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 327ms/step - accuracy: 0.9318 - loss: 0.1968 - val_accuracy: 0.9187 - val_loss: 0.1765
Epoch 7/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 364ms/step - accuracy: 0.9450 - loss: 0.1592 - val_accuracy: 0.9431 - val_loss: 0.1732
Epoch 8/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 311ms/step - accuracy: 0.9323 - loss: 0.1539 - val_accuracy: 0.9431 - val_

In [ ]:
# تقييم النموذج
loss, accuracy = model.evaluate(test_generator)
print(f"✅ دقة النموذج على بيانات الاختبار: {accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.9673 - loss: 0.0973
✅ دقة النموذج على بيانات الاختبار: 97.56%


In [12]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    """ تحميل الصورة وتحويلها إلى الشكل المناسب للنموذج """
    image = Image.open(image_path).convert("RGB")  # تحويل إلى 3 قنوات (RGB)
    image = image.resize((224, 224))  # تغيير الحجم إلى 224x224
    image = np.array(image, dtype=np.float32) / 255.0  # تطبيع القيم بين 0 و 1
    image = np.expand_dims(image, axis=0)  # إضافة بعد جديد ليتناسب مع الإدخال المتوقع (batch_size, 224, 224, 3)
    return image

# تجربة الصورة
image_path = "/content/image (4).png"  # ضع مسار الصورة هنا
processed_image = preprocess_image(image_path)

# التنبؤ باستخدام النموذج
prediction = model.predict(processed_image)

# تفسير النتيجة بناءً على النص الموجود في البيانات
def interpret_prediction(pred):
    """ تحويل التنبؤ الرقمي إلى نص بشري """
    if pred >= 0.5:
        return "مصاب بسرطان الرئة 🩸"
    else:
        return "غير مصاب بالسرطان ✅"

# عرض النتيجة النصية مع احتمالية الإصابة
result = interpret_prediction(prediction[0][0])
print(f"🔍 النتيجة: {result} (احتمال الإصابة: {prediction[0][0] * 100:.2f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
🔍 النتيجة: مصاب بسرطان الرئة 🩸 (احتمال الإصابة: 100.00%)
